# Handling Missing Data

## Trade-Offs in Missing Data Conventions

일반적으로 2가지 접근법이 있다. 
- *mask* 방식
    +  Boolean array 혹은 추가 bit를 통해 표시해 두는 방식
     + 단점: 추가적인 저장을 위해 overhead 발생
- *sentinel value* 방식
    + 특수한 값으로 표시해 두는 방식 (-9999, rare bit pattern, NaN (Not a Number) 등)
    + 단점: 추가적인 연산 logic이 필요


## Missing Data in Pandas
Pandas에서는 두 가지 sentinel value(``NaN``, Python ``None`` object)을 사용한다.

### ``None``: Pythonic missing data

``None``은 Python object이므로 아무곳에 사용될 수는 없으며 arrays of Python objects로 사용될 수 있다.

In [5]:
import numpy as np
import pandas as pd

In [6]:
vals1 = np.array([1, None, 3, 4])
vals1 # None = object type -L> Datatype 전체가 object type로 변경됨.

array([1, None, 3, 4], dtype=object)

``dtype=object`` 즉 Python object의 array임을 말하며 단점은 계산 속도가 느려질 수 있다는 점이다.
(아래 예)


In [7]:
for dtype in ['object', 'int']:
    print("dtype =", dtype)
    %timeit np.arange(1E6, dtype=dtype).sum()
    print()

dtype = object
75.6 ms ± 2.35 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

dtype = int
2.52 ms ± 115 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)



또한 Python object 사용으로 인해 ``sum()`` 이나 ``min()``과 같은 method가 실행될 수 없다.

In [8]:
# vals1.sum()

### ``NaN``: Missing numerical data

``NaN`` (acronym for *Not a Number*)는 특별한 floating-point value이며 표준 IEEE floating-point representation에 따라 대부분 시스템에서 처리된다.

In [36]:
vals2 = np.array([1, np.nan, 3, 4]) 
vals2.dtype
# NaN = folat type! 

dtype('float64')

``NaN``은 부동소수점 소수이므로 연산이 가능하지만 그 결과는 다시 ``NaN``이 된다.

In [10]:
1 + np.nan

nan

In [11]:
0 *  np.nan

nan

따라서 앞서 sum과 같은 array method가 오류로 멈추지는 않지만 단지 NaN이 된다. 

In [37]:
vals2.sum(), vals2.min(), vals2.max()

(nan, nan, nan)

NumPy는 이러한 NaN을 계산에서 배제하는 함수를 제공한다. 

In [13]:
np.nansum(vals2), np.nanmin(vals2), np.nanmax(vals2)

(8.0, 1.0, 4.0)

### NaN and None in Pandas

Pandas에서는 ``NaN`` 과 ``None``이 상황에 따라서 자동적으로 상호호환적으로 처리된다.

In [14]:
pd.Series([1, np.nan, 2, None])

0    1.0
1    NaN
2    2.0
3    NaN
dtype: float64

In [15]:
x = pd.Series(range(2), dtype=int)
x

0    0
1    1
dtype: int32

In [16]:
x[0] = None
x

0    NaN
1    1.0
dtype: float64

## Operating on Null Values

이러한 값의 처리를 위해 Pandas는 다음과 같은 함수가 제공된다. 

- ``isnull()``: Generate a boolean mask indicating missing values
- ``notnull()``: Opposite of ``isnull()``
- ``dropna()``: Return a filtered version of the data
- ``fillna()``: Return a copy of the data with missing values filled or imputed



### Detecting null values
``isnull()`` 과``notnull()``을 사용한다.

In [17]:
data = pd.Series([1, np.nan, 'hello', None])
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [18]:
data.isnull()

0    False
1     True
2    False
3     True
dtype: bool

In [19]:
data[data.notnull()]

0        1
2    hello
dtype: object

### Dropping null values

``dropna()``과 ``fillna()``를 사용할 수 있다.

In [20]:
data.dropna()

0        1
2    hello
dtype: object

In [34]:
data.fillna('false')

a      1.0
b    false
c      2.0
d    false
e      3.0
dtype: object

``DataFrame``의 경우 더 많은 option들이 있다. 

In [21]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


``DataFrame``에서는 행 또는 열 전체를 삭제해야 한다. 

In [22]:
df.dropna()

,0,1,2
1,2.0,3.0,5


In [23]:
df.dropna(axis='columns') # axis=1 과 동일

,2
0,2
1,5
2,6


In [35]:
df.fillna('trash')

,0,1,2,3
0,1.0,trash,2,trash
1,2.0,3.0,5,trash
2,trash,4.0,6,trash


``how``나 ``thresh``를 통해 삭제되는 조건을 줄 수 있다. 

In [24]:
df[3] = np.nan
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [25]:
df.dropna(axis='columns', how='all')

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


``thresh`` parameter는 a minimum number of non-null values 를 지정한다. 

In [26]:
df.dropna(axis='rows', thresh=3)
# 행을 기준으로 n개 미만 입력되면 그 축을 삭제하겠다.

,0,1,2,3
1,2.0,3.0,5,NaN


### Filling null values

``fillna()`` method를 사용할 수 있다.

In [27]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

a    1.0
b    NaN
c    2.0
d    NaN
e    3.0
dtype: float64

In [28]:
data.fillna(0)

a    1.0
b    0.0
c    2.0
d    0.0
e    3.0
dtype: float64

In [29]:
# forward-fill
data.fillna(method='ffill') # NaN 앞에 있는 것으로 채우기
# 만약 첫 번째 원소가 NaN이면?

a    1.0
b    1.0
c    2.0
d    2.0
e    3.0
dtype: float64

In [30]:
# back-fill
data.fillna(method='bfill') # NaN 뒤에 있는 것으로 채우기
# 만약 마지막 원소가 NaN이면?

a    1.0
b    2.0
c    2.0
d    3.0
e    3.0
dtype: float64

``DataFrame``에서는 축 방향을 물론 고려할 수 있다. 

In [31]:
df

,0,1,2,3
0,1.0,NaN,2,NaN
1,2.0,3.0,5,NaN
2,NaN,4.0,6,NaN


In [32]:
df.fillna(method='ffill', axis=1)

,0,1,2,3
0,1.0,1.0,2.0,2.0
1,2.0,3.0,5.0,5.0
2,NaN,4.0,6.0,6.0


Forward fill이 가능하지 않은 element의 경우 NA 값이 유지된다.